In [57]:
# This file is an implementation of the 3D IPM
# Author : Avadesh Meduri
# Date : 1/06/2020

import numpy as np

import IPython
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

from py_motion_planner.ip_motion_planner import IPMotionPlanner

In [58]:
'''
This is an implementation of the inverted pendulum environment to train a 3d dq stepper
'''

class InvertedPendulumEnv:
    
    def __init__(self, h, b, max_step_length, w, no_actions = [11, 9]):
        '''
        Input:
            h : height of the com above the ground at the start of the step
            b : width of the base (distance between the feet)
            max_step_length : max step length allowed
            w : weights for the cost computation
            no_actions : number of discretizations
        '''
        self.g = 9.81
        self.max_leg_length = 0.32
        # maximum accelertion in the z direction by applying force on the ground
        self.max_acc = 7.0
        self.dt = 0.001
        # nominal desired hight of pendulum above the ground (need not be satisfied at all times)
        # is a soft constraint in the qp
        self.h = h
        self.b = b
        # com_offset : the distance betweem center of mass and hip   
        self.com_offset = 0.078
        self.no_steps = 0
        assert len(w) == 3
        self.w = w
        assert (np.linalg.norm([max_step_length[0], self.h - self.com_offset]) < self.max_leg_length)
        assert (np.linalg.norm([max_step_length[1], self.h - self.com_offset]) < self.max_leg_length)
        assert len(no_actions) == 2
        # The co ordinate axis is x : forward and y : sideways walking, z : faces upward
        # This means that left leg is on the positive side of the y axis
        # The addition b is added to accomodate a step length larger than leg length as it may be feasible
        # in high velocity cases.
        self.action_space_x = np.around(np.linspace(-1.2*max_step_length[0], 1.2*max_step_length[0], no_actions[0]), 2)
        # actions to the free side
        if b > 0 :
            self.action_space_ly = np.geomspace(b, 1.5*max_step_length[1] + b, int(2*no_actions[1]/3))
            # actions to the non free side where leg can hit the other leg
            # Y axis actions step length allowed such that robot can't step to the left of the left leg
            # or the right to the right leg (no criss crossing)
            self.action_space_ry = np.linspace(0, b, int(no_actions[1]/3), endpoint = False)
            self.action_space_y = np.around(np.concatenate((self.action_space_ry, self.action_space_ly)), 2)
        else:
            self.action_space_y = np.around(np.linspace(0, max_step_length[1], int(no_actions[1])), 2)
        
        self.t = 0
        self.max_step_height = 0.00
        # QP parameters
        self.delta_t = 0.01
        self.ipmotionplanner = IPMotionPlanner(self.delta_t, self.max_acc)
    
    def integrate_ip_dynamics(self, x_t, u_t, z_acc):
        '''
        This function integrated the ip dynamics for one step using euler integration scheme
        Input:
            x_t : state at time step t (cx, cy, cz, cxd, cyd, czd)
            u_t : Cop location at time step t (ux, uy, uz)
            z_acc : acceleration in z direction (control input to increase height)
        '''
        
        x_t_1 = np.zeros(6)
        x_t_1[0:3] = np.add(x_t[0:3], x_t[3:]*self.dt)
        x_t_1[3] = x_t[3] + ((z_acc + self.g)*(x_t[0] - u_t[0])/(x_t[2] - u_t[2]))*self.dt
        x_t_1[4] = x_t[4] + ((z_acc + self.g)*(x_t[1] - u_t[1])/(x_t[2] - u_t[2]))*self.dt
        x_t_1[5] = x_t[5] + z_acc*self.dt

        return x_t_1
    
    def reset_env(self, x0, v_des, epi_time):
        '''
        Resets environment for a new episode
        Input:
            x0 : initial state of the system [x, y, z, xd, yd, zd]
            v_des : desired velocity [xd_des, yd_des]
            epi_time : episode time
        '''
        assert len(x0) == 5
        self.t = 0
        # [x, y, z, xd, yd, zd, ux, uy, uz, n]
        self.sim_data = np.zeros((10, int(epi_time/self.dt)+1))
        self.no_steps = 0
        assert (len(v_des) == 2)
        self.v_des = v_des
        assert (np.linalg.norm([x0[0], x0[2] - self.com_offset]) < self.max_leg_length)
        assert (np.linalg.norm([x0[1], x0[2] - self.com_offset]) < self.max_leg_length)
        self.sim_data[:,0][0:5] = x0
        self.sim_data[:,0][7] = -self.b/2 # right leg on the ground
        self.sim_data[:,0][9] = 1 # determines which leg is on the ground (1 is right leg)
        
        processed_state = np.zeros(8)
        processed_state[0:6] = np.take(self.sim_data[:,0], [0, 1, 2, 3, 4, 9])
        processed_state[0:3] -= self.sim_data[:,self.t][6:9] # shifting origin to u
        processed_state[6:8] = self.v_des    
        
        return processed_state
    
    
    def step_env(self, u, step_time):
        '''
        This function simulates the environment for one foot step
        Input:
            u : next step location
            step_time : duration of after which step is taken [ux_index, uy_index, uz (value)]
        '''
    
        assert u[2] < self.max_step_height + 0.000001
        x , xd, acc = self.ipmotionplanner.generate_force_trajectory(self.sim_data[:,self.t][2], \
                                                             self.sim_data[:,self.t][8] + u[2], step_time, self.h)
        acc = np.repeat(acc, int(self.delta_t/self.dt))
        for i in range(int(step_time/self.dt)-1):
            self.sim_data[:,self.t+1][0:6] = self.integrate_ip_dynamics(self.sim_data[:,self.t][0:6], \
                                                                   self.sim_data[:,self.t][6:9], acc[i]) 
            self.sim_data[:,self.t+1][6:9] = self.sim_data[:,self.t][6:9] #u
            self.sim_data[:,self.t+1][9] = self.sim_data[:,self.t][9] #n
            self.t += 1

        self.sim_data[:,self.t][6] += self.action_space_x[u[0]]
        self.sim_data[:,self.t][7] += self.sim_data[:,self.t][9]*self.action_space_y[u[1]]
        self.sim_data[:,self.t][8] += u[2]
        self.sim_data[:,self.t][9] = -1*self.sim_data[:,self.t][9]
        
        ## modifying state that is returned is such that the origin is u0 instead of the global origin
        ## This ensures that the state x[0] is bounded by the maximum leg size while collecting data
        processed_state = np.zeros(8)
        processed_state[0:6] = np.take(self.sim_data[:,self.t], [0, 1, 2, 3, 4, 9])
        processed_state[0:3] -= self.sim_data[:,self.t][6:9] # shifting origin to u
        processed_state[6:8] = self.v_des    
        
        if not self.isdone():
            self.no_steps += 1
        
        return np.round(processed_state, 2), self.compute_cost(), self.isdone()
    
    def isdone(self):
        '''
        Checks if the kinematic constraints are violated
        '''
        # Computing the hip location
        hip = self.sim_data[:,self.t][0:2].copy()
        hip[1] -= self.sim_data[:,self.t][9]*(self.b/2.0)
        tmp = np.linalg.norm(hip - self.sim_data[:,self.t][6:8])
        h = self.sim_data[:,self.t][2] - self.sim_data[:,self.t][8] - self.com_offset
        current_leg_length = np.linalg.norm([tmp, h])
        if current_leg_length > self.max_leg_length:
            return True
        else:
            return False
        
    def compute_cost(self):
        '''
        Computes cost which is distance between the hip(closest hip depending on which foot is on the ground)
        and the foot + velocity of the center of mass + 1 if step length not equal to zero (after taking into
        account the offset) + 100 if episode terminates (kinematics constraints are violated)
        '''
        hip = self.sim_data[:,self.t][0:3].copy()
        hip[1] += -1*self.sim_data[:,self.t][9]*(self.b/2) # -1 is to match co ordinate axis
        u = self.sim_data[:,self.t][6:9].copy()
        cost = self.w[0]*(abs(hip - u)[0]) + self.w[0]*(abs(hip - u)[1])
        if self.isdone():
            cost += 100
        cost += self.w[1]*(abs(self.sim_data[:,self.t][3] - self.v_des[0]) \
                               + abs(self.sim_data[:,self.t][4] - self.v_des[1]))
        
        cost += self.w[2]*(abs(np.round(self.sim_data[:,self.t][6] - self.sim_data[:,self.t - 5][6], 2)))
        cost += abs(self.w[2]*(abs(np.round(self.sim_data[:,self.t][7] - self.sim_data[:,self.t - 5][7], 2)) - self.b))
        cost += self.w[2]*(abs(np.round(self.sim_data[:,self.t][8] - self.sim_data[:,self.t - 5][8], 2)))
    
        return cost
    
    def random_action(self):
        '''
        Genarates random action
        '''
        action_x = np.random.randint(len(self.action_space_x))
        action_y = np.random.randint(len(self.action_space_y))
        action_z = self.max_step_height*(np.random.rand() - 0.5)
        
        return [action_x, action_y, action_z]

    def show_episode(self, freq):
        '''
        Shows animation
        Input :
            freq : frame rate
        '''
        sim_data = self.sim_data[:,::freq]
        
        fig = plt.figure()
        ax = plt.axes(xlim=(-1, 1), ylim=(-1, 1))
        text_str = "top view of IPM"
        base, = ax.plot([], [], lw=3, color = 'blue')
        leg, = ax.plot([], [], lw=3, color = 'pink')
        com, = ax.plot([], [], 'o', color='red')
        foot, = ax.plot([], [], 'o', color='green')

        
        def init():
            base.set_data([], [])
            leg.set_data([], [])
            com.set_data([], [])
            foot.set_data([], [])
            return base, leg, com, foot
        
        def animate(i):
            x_com = sim_data[:,i][0]
            y_com = sim_data[:,i][1]
            u_x = sim_data[:,i][6]
            u_y = sim_data[:,i][7]
            n = sim_data[:,i][9]
            
            base.set_data([x_com, x_com], [y_com - self.b/2, y_com + self.b/2])
            leg.set_data([x_com, u_x], [y_com - n*(self.b/2), u_y])
            com.set_data([x_com], [y_com])
            foot.set_data([u_x], [u_y])
            return base,leg, com, foot
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(0.05, 0.95, text_str, transform=ax.transAxes, fontsize=15,
        verticalalignment='top', bbox=props)
        
        anim = FuncAnimation(fig, animate, init_func=init,
                                       frames=np.shape(sim_data)[1], interval=25, blit=True)
        plt.grid()
        plt.close(fig)
        plt.close(anim._fig)
        IPython.display.display_html(IPython.core.display.HTML(anim.to_html5_video()))
        
    def show_episode_side(self, freq):
        '''
        shows animation from the side view
        Input:
            freq : frame rate
        '''
        sim_data = self.sim_data[:,::freq]

        fig = plt.figure()
        ax = plt.axes(xlim=(-2, 2), ylim=(-0.2, 0.5))
        text_str = "side view (xz plane)"
        leg, = ax.plot([], [], lw=4)
        body, = ax.plot([], [], lw=4)
        head, = ax.plot([], [], 'o', color='green')
        com, = ax.plot([], [], 'o', color='red')
        foot, = ax.plot([], [], 'o', color='pink')
        
        def init():
            leg.set_data([], [])
            body.set_data([], [])
            head.set_data([], [])
            com.set_data([], [])
            foot.set_data([], [])
            
            return leg, body, head, com, foot
        
        def animate(i):
            x = sim_data[:,i][0]
            y = sim_data[:,i][2]
            ux = sim_data[:,i][6]
            uy = sim_data[:,i][8]
            
            leg.set_data([ux,x], [uy,y - self.com_offset])
            com.set_data([x, y])
            body.set_data([x, x], [y, y - self.com_offset])
            head.set_data([x, y])
            foot.set_data([ux, uy])

            return leg, com, body, head, foot
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(0.05, 0.95, text_str, transform=ax.transAxes, fontsize=15,
        verticalalignment='top', bbox=props)
        
        anim = FuncAnimation(fig, animate, init_func=init,
                                       frames=np.shape(sim_data)[1], interval=25, blit=True)

        plt.close(fig)
        plt.close(anim._fig)
        IPython.display.display_html(IPython.core.display.HTML(anim.to_html5_video()))
    
        

In [72]:
show = False

if show:
    no_steps = 10 ## simulates 10 steps taken by the inverted pendulum
    step_time = 0.2 ## duration after which step is taken by pendulum

    env = InvertedPendulumEnv(0.28, 0.13, [0.0, 0.2], [0, 0, 1], [1,9])
    state = env.reset_env([0.0,0.0, 0.28, 0.0, 0.0], [0.0, 0], no_steps*step_time)
    for t in range(no_steps):
        action = env.random_action()
        if t == 0:
            action = [0, 0, 0.00]
        elif t == 1:
            action = [0, 0, 0.00]
        elif t == 2:
            action = [0, 0, 0.00]
#         elif t == 3:
#             action = [5, 5, 0.00]
#         elif t == 4:
#             action = [5, 3, 0.00]
#         elif t == 5:
#             action = [5, 4, 0.00]
                
        next_state, cost, done = env.step_env(action, step_time)
        print(state[3:5], cost)
        state = next_state
        if done:
            break
    env.show_episode(10)
#     env.show_episode_side(5)
    

[0.6 0. ] 0.24
[1.06 0.  ] 0.24
[1.09 0.  ] 0.24
[0.72 0.  ] 100.05


Your browser does not support the video tag.

In [67]:
np.sqrt(0.28/9.81)*0.2*np.power(np.e, (1/np.sqrt(0.28/9.81))*0.3)

0.19950965243968136